In [33]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from time import time
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import MiniBatchKMeans
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Données de la première partie récupérée
mnist = fetch_openml('mnist_784', version=1)
mnist['target'] = mnist['target'].astype('uint8')
X = mnist['data']
y = mnist['target']

## 1. Résultats de RandomForest SANS la réduction de la dimensionnalité des données

In [3]:
# Division de la base de données
X_train, y_train = X[:60000,:], y[:60000]
X_test, y_test = X[60000:,:], y[60000:]

In [12]:
# Classification avec RandomForest
start = time()
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)
end = time()

exec_time = end - start
print(f'{round(exec_time, 2)}')

31.0


In [14]:
# Prédiction sur X_test
pred = clf.predict(X_test)

In [19]:
# Evaluation du score du modèle
print(f'Evaluation avec la fonction du RandomForest: {clf.score(X_test, y_test)}')
print(f'Evaluation avec accuracy_score: {metrics.accuracy_score(pred, y_test)}')

Evaluation avec la fonction du RandomForest: 0.9691
Evaluation avec accuracy_score: 0.9691


## 2. Résultats de RandomForest AVEC la réduction de la dimensionnalité des données

In [25]:
# Déclaration modèle PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_train)

In [26]:
# Classification avec RandomForest
start = time()
clf = clf.fit(X_pca, y_train)
end = time()

In [29]:
# Calcul du temps de calcul
exec_time = end - start
print(f'{round(exec_time, 2)}')

73.27


Le temps d'apprentissage est plus long que pour le premier apprentissage.   
Il est environ deux fois plus important que lors du premier apprentissage.  

On pouvait s'attendre à ça car on a des données formatés différemment.

In [30]:
X_pca_test = pca.transform(X_test)

In [31]:
# Prédiction sur X_pca_test
pred_pca = clf.predict(X_pca_test)

In [32]:
# Evaluation du score du modèle
print(f'Evaluation avec la fonction du RandomForest: {clf.score(X_pca_test, y_test)}')
print(f'Evaluation avec accuracy_score: {metrics.accuracy_score(pred_pca, y_test)}')

Evaluation avec la fonction du RandomForest: 0.9468
Evaluation avec accuracy_score: 0.9468


On peut conclure que l'application du PCA a fait doubler le temps d'apprentissage du modèle et le score de précision du modèle est moins bon que lorsque l'on utilise pas PCA.

## 3. Résultats de Softmax SANS la réduction de la dimensionnalité des données

In [44]:
# Classification avec une régression logistique
start = time()
clf = LogisticRegression(random_state=0, max_iter=100)
clf.fit(X_train, y_train)
end = time()

/home/thomasc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
# Calcul du temps de calcul
exec_time = end - start
print(f'{round(exec_time, 2)}')

16.09


In [48]:
# Prédiction sur X_test
pred_log = clf.predict(X_test)

In [49]:
# Evaluation du score du modèle
print(f'Evaluation avec la fonction du RandomForest: {clf.score(X_test, y_test)}')
print(f'Evaluation avec accuracy_score: {metrics.accuracy_score(pred_log, y_test)}')

Evaluation avec la fonction du RandomForest: 0.9255
Evaluation avec accuracy_score: 0.9255


## 4. Résultats de Softmax AVEC la réduction de la dimensionnalité des données

In [55]:
# Application du PCA
pca = PCA(n_components=0.95)
X_log_pca = pca.fit_transform(X_train)

In [56]:
# Classification avec une régression logistique avec PCA
start = time()
clf = clf.fit(X_log_pca, y_train)
end = time()

/home/thomasc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
# Calcul du temps de calcul
exec_time = end - start
print(f'{round(exec_time, 2)}')

5.29


Le temps d'apprentissage est vraiment plus rapide après application du `PCA`, 3 fois plus rapide.

In [58]:
X_log_pca_test = pca.transform(X_test)

In [59]:
# Prédiction sur X_pca_test
pred_log_pca = clf.predict(X_log_pca_test)

In [60]:
# Evaluation du score du modèle
print(f'Evaluation avec la fonction du RandomForest: {clf.score(X_log_pca_test, y_test)}')
print(f'Evaluation avec accuracy_score: {metrics.accuracy_score(pred_log_pca, y_test)}')

Evaluation avec la fonction du RandomForest: 0.9201
Evaluation avec accuracy_score: 0.9201


On voit que le score des deux modèles sont quasiment similaires.  
En revanche le temps de calcul était vraiment plus rapide après application du `PCA`.

#### On voit que l'application n'est pas toujours bénéfique.   

#### Pour le cas du `RandomForest` l'application du `PCA` n'est pas bénéfique ni pour le temps d'apprentissage ni pour le score du modèle.   
#### En revanche pour `Softmax` l'application du `PCA` est bénéfique pour le temps d'apprentissage et cela n'affecte pas le score.